In [2]:
import pandas as pd
from datetime import datetime
import numpy as np
import time

In [3]:
# Latest Category
category_main = pd.read_excel("checkpoint_files/tracking_with_updated_category_daily_activity.xlsx")


# get only whatsapp and category
donor_category = category_main[['Whatsapp','Kategori']]

In [4]:
database_all = pd.read_excel("Database.xlsx")
database_all['Kategori'].value_counts()

Kategori
AKTIF    43660
Aktif     1063
LOYAL      145
Loyal       37
Name: count, dtype: int64

In [5]:
category_update = database_all[['Whatsapp','Kategori']].merge(donor_category,how='left', on='Whatsapp')
category_update

,Whatsapp,Kategori_x,Kategori_y
0,62881082355438,AKTIF,AKTIF
1,6285753301575,AKTIF,AKTIF
2,6282236899125,AKTIF,AKTIF
3,628226357000,AKTIF,INVALID
4,628561889392,LOYAL,LOYAL
...,...,...,...
44900,628281723403,AKTIF,NaN
44901,6282368483535,AKTIF,NaN
44902,6285397031692,AKTIF,NaN
44903,62817161743,AKTIF,NaN


In [43]:
# def update_category(df):
#     # if Kategori_y isna take Kategori_x
#     if pd.isna(df['Kategori_y']):
#         return df['Kategori_x']
    
#     # if Kategori_x and Kategori_y are different take Kategori_y
#     elif df['Kategori_x'] != df['Kategori_y']:
#         return df['Kategori_y']
    
#     # if both are the same take Kategori_x
#     else:
#         return df['Kategori_x']

# category_update['Kategori'] = category_update.apply(update_category, axis=1)

# Start Here

In [56]:
category_update = pd.read_excel("checkpoint_files/category_each_donor.xlsx")

In [46]:
# daily_1097 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_1097.xlsx")
# daily_2056_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2056 (BONDING).xlsx")
# daily_2056_maintain = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2056 (MAINTAIN).xlsx")
# daily_2057_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2057 (BONDING).xlsx")
# daily_2057_maintain = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2057 (MAINTAIN).xlsx")
# daily_0046 = pd.read_excel("../data_blast/daily_activity/filled/new/0046_DA_Shania.xlsx")
# daily_0047 = pd.read_excel("../data_blast/daily_activity/filled/new/0047_DA_Shania.xlsx")
# daily_0041 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0041.xlsx")
# daily_0051 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_0051.xlsx")
daily_2031 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_WA_2031.xlsx")
daily_2032 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_wa_2032.xlsx")
daily_7778 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_7778.xlsx")


# daily_2056 = pd.concat([daily_2056_bonding, daily_2056_maintain])
# daily_2057 = pd.concat([daily_2057_bonding, daily_2057_maintain])


daily_activity_all = pd.concat([daily_2031,daily_2032,daily_7778])
daily_activity_all = daily_activity_all.dropna(subset=['Whatsapp'])

c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
update_category = category_update.merge(daily_activity_all[['Whatsapp','Next Action', 'Tanggal 3']], how='left', on='Whatsapp')

# standarize the value, for date and prioritas
update_category['Tanggal 3'] = pd.to_datetime(update_category['Tanggal 3'], errors='coerce')
update_category.loc[update_category['Next Action'] == "Prioirtas 3", "Next Action"] = "Prioritas 3"

# rename columns
# update_category = update_category.rename(columns={"Kategori":"kategori_update",
#                                                   "Next Action":"kategori_prioritas",
#                                                   "Tanggal 3":"terakhir_interaksi"})

In [49]:
update_category

,Whatsapp,kategori_update,kategori_prioritas,terakhir_interaksi,Next Action,Tanggal 3
0,62881082355438,Aktif,NaN,NaT,NaN,NaT
1,6285753301575,Aktif,NaN,NaT,NaN,NaT
2,6282236899125,Aktif,NaN,NaT,NaN,NaT
3,628226357000,Invalid,NaN,NaT,NaN,NaT
4,628561889392,Loyal,NaN,NaT,NaN,NaT
...,...,...,...,...,...,...
44438,6281371988248,Aktif,NaN,NaT,NaN,NaT
44439,628126512796,Aktif,NaN,NaT,NaN,NaT
44440,6285641084108,Aktif,NaN,NaT,NaN,NaT
44441,6281395480043,Aktif,NaN,NaT,NaN,NaT


In [65]:
# def category_update(df):
#     # if Kategori_y isna take Kategori_x
#     if pd.isna(df['Next Action']):
#         return df['kategori_prioritas']
    
#     # if Kategori_x and Kategori_y are different take Kategori_y
#     elif df['kategori_prioritas'] != df['Next Action']:
#         return df['Next Action']
    
#     # if both are the same take Kategori_x
#     else:
#         return df['kategori_prioritas']

update_category['kategori_prioritas'] = update_category['kategori_prioritas'].fillna(update_category['Next Action'])
update_category['terakhir_interaksi'] = update_category['terakhir_interaksi'].fillna(update_category['Tanggal 3'])

In [66]:
# update category when on the daily activity is eliminasi or invalid then return that value
update_category['kategori_update'] = np.where(update_category['kategori_prioritas'].isin(["Eliminasi", "Invalid"]), update_category['kategori_prioritas'], update_category['kategori_update'])

# update_category.loc[update_category['kategori_update'] == "AKTIF", "kategori_update"] = "Aktif"
# update_category.loc[update_category['kategori_update'] == "INVALID", "kategori_update"] = "Invalid"
# update_category.loc[update_category['kategori_update'] == "LOYAL", "kategori_update"] = "Loyal"
# update_category['kategori_update'].apply(lambda x: x if x in ["Eliminasi", "Invalid"] else )

In [70]:
update_category = update_category[['Whatsapp','kategori_update','kategori_prioritas','terakhir_interaksi']]

In [72]:
update_category.to_excel("checkpoint_files/category_each_donor.xlsx", index=False)